# Home Credit Analysis

Goal of this notebook is to answer:

- What factors will affect how much an individual is approved for?
- How does a previous application affect your loan amount?
- Which loan purpose, increase or decrease the amount difference?
- What start time of application, increase or decrease the amount difference? 

Data: https://www.kaggle.com/c/home-credit-default-risk/data

# Importing Data

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from matplotlib import pyplot as pyplot

from google.colab import drive
drive.mount('/content/drive')
drive_dir = '/content/drive/Shared drives/Project 4 (MATH 3439)/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Loading the Data

In [ ]:
#df1_train = pd.read_csv(drive_dir + 'application_train.csv.zip')
#df2_test = pd.read_csv(drive_dir + 'application_test.csv.zip')

In [ ]:
sample_df_instalpay = pd.read_csv(drive_dir + 'installments_payments_mini.csv')

In [ ]:
sample_df_poscashb = pd.read_csv(drive_dir + 'POS_CASH_balance_mini.csv')

In [ ]:
sample_df_bureau = pd.read_csv(drive_dir + 'bureau.csv_mini.csv')

In [ ]:
sample_df_bureaubal = pd.read_csv(drive_dir + 'bureau_balance_mini.csv')

In [ ]:
sample_df_creditcb = pd.read_csv(drive_dir + 'credit_card_balance_mini.csv')

In [ ]:
sample_df_app = pd.read_csv(drive_dir + 'previous_application_mini.csv')

In [ ]:
result = pd.read_csv(drive_dir + 'result.csv')

# Data Cleaning

### Intallment Pay

In [ ]:
sample_df_instalpay.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,2599877,174840,1.0,7,-2296.0,-2292.0,10716.210,10716.210
1,1102197,138433,0.0,85,-1396.0,-1396.0,2250.000,2250.000
2,2012087,121332,1.0,7,-240.0,-242.0,25584.525,25584.525
3,2792257,127624,1.0,4,-1502.0,-1510.0,3758.670,3758.670
4,1560899,168735,2.0,10,-187.0,-180.0,32495.985,32495.985


In [ ]:
sample_df_instalpay.shape

(880000, 8)

Checking for duplicates.

In [ ]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = sample_df_instalpay[sample_df_instalpay.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
        SK_ID_PREV  SK_ID_CURR  ...  AMT_INSTALMENT  AMT_PAYMENT
159395     2149338      111259  ...        28730.25     28730.25

[1 rows x 8 columns]


In [ ]:
sample_df_instalpay['SK_ID_CURR'].nunique()

90974

In [ ]:
sample_df_instalpay = sample_df_instalpay.drop(['SK_ID_PREV'], axis=1)

In [ ]:
for c in sample_df_instalpay.columns:
  num_missing = sample_df_instalpay[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_instalpay.shape[0]:.2f}%) missing values')

### POS CASH

In [ ]:
sample_df_poscashb.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1239342,216180,-18,24.0,19.0,Active,0,0
1,2179568,397416,-56,7.0,4.0,Active,0,0
2,2501225,257924,-4,12.0,10.0,Active,0,0
3,2023169,170080,-11,18.0,11.0,Active,0,0
4,1374439,148961,-21,12.0,5.0,Active,0,0


In [ ]:
for c in sample_df_poscashb.columns:
  num_missing = sample_df_poscashb[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_poscashb.shape[0]:.2f}%) missing values')

CNT_INSTALMENT: 1656 (0.19%) missing values
CNT_INSTALMENT_FUTURE: 1657 (0.19%) missing values


In [ ]:
sample_df_poscashb = sample_df_poscashb.drop(['SK_ID_PREV'], axis=1)

In [ ]:
sample_df_poscashb = sample_df_poscashb.dropna(how='any')

In [ ]:
for c in sample_df_poscashb.columns:
  num_missing = sample_df_poscashb[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_poscashb.shape[0]:.2f}%) missing values')

### Bureau

In [ ]:
sample_df_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,406090,5197457,Closed,currency 1,-603,0,-453.0,-421.0,NaN,0,118656.0,NaN,NaN,0.0,Consumer credit,-414,0.000
1,171262,5811086,Active,currency 1,-291,0,-261.0,NaN,NaN,0,202500.0,181287.0,0.00,0.0,Credit card,-233,NaN
2,167780,5717336,Active,currency 1,-205,0,344.0,NaN,0.00,0,297000.0,212571.0,0.00,0.0,Consumer credit,-15,6120.000
3,249764,5212718,Active,currency 1,-1223,0,NaN,NaN,0.00,0,135000.0,115348.5,19650.51,0.0,Credit card,-18,13500.000
4,164484,5253659,Closed,currency 1,-827,0,-274.0,-273.0,6064.29,0,98685.0,0.0,0.00,0.0,Consumer credit,-271,8657.325


In [ ]:
for c in sample_df_bureau.columns:
  num_missing = sample_df_bureau[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_bureau.shape[0]:.2f}%) missing values')

DAYS_CREDIT_ENDDATE: 54646 (6.21%) missing values
DAYS_ENDDATE_FACT: 326730 (37.13%) missing values
AMT_CREDIT_MAX_OVERDUE: 579851 (65.89%) missing values
AMT_CREDIT_SUM: 4 (0.00%) missing values
AMT_CREDIT_SUM_DEBT: 131614 (14.96%) missing values
AMT_CREDIT_SUM_LIMIT: 306648 (34.85%) missing values
AMT_ANNUITY: 610646 (69.39%) missing values


- DAYS_CREDIT_ENDDATE: Remaining duration of CB credit (in days) at the time of application in Home Credit 
- DAYS_ENDDATE_FACT: Days since CB credit ended at the time of application in Home Credit (only for closed credit) 
- AMT_CREDIT_MAX_OVERDUE: Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample) 
- AMT_CREDIT_SUM_DEBT: Current debt on Credit Bureau credit 
- AMT_CREDIT_SUM_LIMIT: Current credit limit of credit card reported in Credit Bureau 
- AMT_ANNUITY: Annuity of the Credit Bureau credit

In [ ]:
sample_df_bureau = sample_df_bureau.fillna(0)

In [ ]:
sample_df_bureau = sample_df_bureau.drop(['SK_ID_BUREAU','CREDIT_CURRENCY'], axis=1)

In [ ]:
sample_df_bureau.shape

(880000, 15)

In [ ]:
for c in sample_df_bureau.columns:
  num_missing = sample_df_bureau[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_bureau.shape[0]:.2f}%) missing values')

### Bureau Balance

In [ ]:
sample_df_bureaubal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5262488,-28,X
1,6021653,-87,0
2,6020603,-44,C
3,5726348,-5,0
4,5004181,-23,0


In [ ]:
for c in sample_df_bureaubal.columns:
  num_missing = sample_df_bureaubal[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_bureaubal.shape[0]:.2f}%) missing values')

### Credit Card

In [ ]:
sample_df_creditcb.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1730221,401839,-25,0.000,225000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
1,2657934,366032,-33,135972.405,135000,0.0,0.0,0.0,0.0,6750.000,6750.0,6750.000,131812.695,135972.405,135972.405,0.0,0,0.0,0.0,44.0,Active,0,0
2,2411238,257442,-4,181374.750,180000,0.0,0.0,0.0,0.0,9096.480,9900.0,9900.000,172581.480,180045.855,180045.855,0.0,0,0.0,0.0,17.0,Active,0,0
3,1698374,353507,-4,468008.640,450000,24750.0,24750.0,0.0,0.0,22285.305,22500.0,22500.000,447956.460,463213.890,463213.890,1.0,1,0.0,0.0,31.0,Active,0,0
4,1214346,322324,-87,72486.135,90000,0.0,0.0,0.0,0.0,4500.000,7200.0,7200.000,69947.055,72486.135,72486.135,0.0,0,0.0,0.0,14.0,Active,0,0
5,2000272,263926,-23,130329.675,135000,0.0,0.0,0.0,0.0,6750.000,6750.0,6750.000,125180.595,130329.675,130329.675,0.0,0,0.0,0.0,51.0,Active,0,0
6,1141492,446669,-5,253425.960,270000,0.0,0.0,0.0,0.0,13434.345,14400.0,1173.465,250130.970,253425.960,253425.960,0.0,0,0.0,0.0,4.0,Active,0,0
7,1891392,441485,-67,111838.455,180000,0.0,0.0,0.0,0.0,10800.000,10800.0,10800.000,107297.865,111838.455,111838.455,0.0,0,0.0,0.0,38.0,Active,0,0
8,2140631,249615,-13,0.000,270000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
9,2037377,289902,-1,131294.340,135000,0.0,0.0,0.0,0.0,6774.660,6975.0,6975.000,128518.020,131294.340,131294.340,0.0,0,0.0,0.0,11.0,Active,0,0


In [ ]:
sample_df_creditcb.shape

(880000, 23)

In [ ]:
sample_df_creditcb = sample_df_creditcb.dropna(how='any')

In [ ]:
sample_df_creditcb = sample_df_creditcb.drop(['SK_ID_PREV','AMT_DRAWINGS_OTHER_CURRENT','AMT_DRAWINGS_POS_CURRENT','CNT_DRAWINGS_OTHER_CURRENT','CNT_DRAWINGS_POS_CURRENT'], axis=1)

In [ ]:
sample_df_creditcb.shape

(683150, 18)

In [ ]:
for c in sample_df_creditcb.columns:
  num_missing = sample_df_creditcb[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_creditcb.shape[0]:.2f}%) missing values')

### Previous Application

In [ ]:
sample_df_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2138331,249458,Consumer loans,18324.045,184275.00,200088.0,0.00,184275.00,WEDNESDAY,14,Y,1,0.000000,NaN,NaN,XAP,Approved,-418,Cash through the bank,XAP,Unaccompanied,Repeater,Audio/Video,POS,XNA,Country-wide,2500,Consumer electronics,12.0,low_action,POS household without interest,365243.0,-387.0,-57.0,-57.0,-50.0,0.0
1,1989262,446664,Consumer loans,6990.615,52155.00,49684.5,10431.00,52155.00,WEDNESDAY,12,Y,1,0.188975,NaN,NaN,XAP,Approved,-1769,Cash through the bank,XAP,Family,New,Consumer Electronics,POS,XNA,Regional / Local,260,Consumer electronics,8.0,middle,POS household with interest,365243.0,-1738.0,-1528.0,-1558.0,-1555.0,0.0
2,2007043,170000,Consumer loans,5825.205,25192.44,29763.0,1.44,25192.44,WEDNESDAY,12,Y,1,0.000053,NaN,NaN,XAP,Approved,-849,Cash through the bank,XAP,Family,Repeater,Photo / Cinema Equipment,POS,XNA,Regional / Local,135,Consumer electronics,6.0,high,POS household with interest,365243.0,-818.0,-668.0,-668.0,-656.0,0.0
3,2392160,335249,Cash loans,NaN,0.00,0.0,NaN,NaN,MONDAY,13,Y,1,NaN,NaN,NaN,XNA,Canceled,-66,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,0,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
4,2808823,193643,Cash loans,19698.120,216000.00,237384.0,NaN,216000.00,THURSDAY,8,Y,1,NaN,NaN,NaN,XNA,Refused,-607,Cash through the bank,HC,NaN,Repeater,XNA,Cash,x-sell,Country-wide,58,Connectivity,18.0,high,Cash X-Sell: high,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for c in sample_df_app.columns:
  num_missing = sample_df_app[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_app.shape[0]:.2f}%) missing values')

AMT_ANNUITY: 195166 (22.18%) missing values
AMT_DOWN_PAYMENT: 468390 (53.23%) missing values
AMT_GOODS_PRICE: 201892 (22.94%) missing values
RATE_DOWN_PAYMENT: 468390 (53.23%) missing values
RATE_INTEREST_PRIMARY: 876845 (99.64%) missing values
RATE_INTEREST_PRIVILEGED: 876845 (99.64%) missing values
NAME_TYPE_SUITE: 432011 (49.09%) missing values
CNT_PAYMENT: 195164 (22.18%) missing values
PRODUCT_COMBINATION: 190 (0.02%) missing values
DAYS_FIRST_DRAWING: 352762 (40.09%) missing values
DAYS_FIRST_DUE: 352762 (40.09%) missing values
DAYS_LAST_DUE_1ST_VERSION: 352762 (40.09%) missing values
DAYS_LAST_DUE: 352762 (40.09%) missing values
DAYS_TERMINATION: 352762 (40.09%) missing values
NFLAG_INSURED_ON_APPROVAL: 352762 (40.09%) missing values


In [ ]:
sample_df_app = sample_df_app.drop(['RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED','SK_ID_PREV',], axis=1)

In [ ]:
sample_df_app = sample_df_app.dropna(how='any')

In [ ]:
sample_df_app.shape

(228805, 34)

#Reducing Memory Usage

### Installments_payments

Checking for Memory usage and `dtypes`

In [ ]:
sample_df_instalpay.memory_usage() / 10**6

Index                     0.000128
SK_ID_CURR                7.040000
NUM_INSTALMENT_VERSION    7.040000
NUM_INSTALMENT_NUMBER     7.040000
DAYS_INSTALMENT           7.040000
DAYS_ENTRY_PAYMENT        7.040000
AMT_INSTALMENT            7.040000
AMT_PAYMENT               7.040000
dtype: float64

In [ ]:
sample_df_instalpay.memory_usage().sum() / 10**6

49.280128

In [ ]:
sample_df_instalpay.dtypes

SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object

In [ ]:
int_cols = sample_df_instalpay.select_dtypes(int).columns
int_cols

Index(['SK_ID_CURR', 'NUM_INSTALMENT_NUMBER'], dtype='object')

In [ ]:
for c in int_cols:
    print(f'{c}: min = {sample_df_instalpay[c].min()}, max = {sample_df_instalpay[c].max()}')

SK_ID_CURR: min = 100002, max = 199999
NUM_INSTALMENT_NUMBER: min = 1, max = 244


In [ ]:
big_int_cols = ['SK_ID_CURR']

for col in big_int_cols:
    sample_df_instalpay[col] = sample_df_instalpay[col].astype(np.uint32)

In [ ]:
sample_df_instalpay['NUM_INSTALMENT_NUMBER'] = sample_df_instalpay['NUM_INSTALMENT_NUMBER'].astype(np.uint8)

In [ ]:
float_cols = sample_df_instalpay.select_dtypes(float).columns
float_cols

Index(['NUM_INSTALMENT_VERSION', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')

In [ ]:
for c in float_cols:
    print(f'{c}: min = {sample_df_instalpay[c].min()}, max = {sample_df_instalpay[c].max()}')

NUM_INSTALMENT_VERSION: min = 0.0, max = 40.0
DAYS_INSTALMENT: min = -2922.0, max = -2.0
DAYS_ENTRY_PAYMENT: min = -3071.0, max = -2.0
AMT_INSTALMENT: min = 0.0, max = 3473582.895
AMT_PAYMENT: min = 0.0, max = 3473582.895


In [ ]:
sample_df_instalpay['NUM_INSTALMENT_VERSION'] = sample_df_instalpay['NUM_INSTALMENT_VERSION'].astype(np.int8)

In [ ]:
small_float_cols = ['DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT']
big_float_cols = ['AMT_INSTALMENT','AMT_PAYMENT']

for col in small_float_cols:
    sample_df_instalpay[col] = sample_df_instalpay[col].astype(np.int16)
    
for col in big_float_cols:
    sample_df_instalpay[col] = sample_df_instalpay[col].astype(np.int32)

In [ ]:
sample_df_instalpay.memory_usage().sum() / 10**6

15.840128

### POS_CASH_balance.csv.zip

In [ ]:
sample_df_poscashb.memory_usage() / 10**6

Index                    7.026728
SK_ID_CURR               7.026728
MONTHS_BALANCE           7.026728
CNT_INSTALMENT           7.026728
CNT_INSTALMENT_FUTURE    7.026728
NAME_CONTRACT_STATUS     7.026728
SK_DPD                   7.026728
SK_DPD_DEF               7.026728
dtype: float64

In [ ]:
sample_df_poscashb.memory_usage().sum() / 10**6

56.213824

In [ ]:
sample_df_poscashb.dtypes

SK_ID_CURR                 int64
MONTHS_BALANCE             int64
CNT_INSTALMENT           float64
CNT_INSTALMENT_FUTURE    float64
NAME_CONTRACT_STATUS      object
SK_DPD                     int64
SK_DPD_DEF                 int64
dtype: object

In [ ]:
sample_df_poscashb['NAME_CONTRACT_STATUS'].unique()

array(['Active', 'Completed', 'Demand', 'Signed', 'Approved',
       'Returned to the store'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
contract_le = LabelEncoder()
contract_le.fit(sample_df_poscashb['NAME_CONTRACT_STATUS'])
sample_df_poscashb['NAME_CONTRACT_STATUS'] = contract_le.transform(sample_df_poscashb['NAME_CONTRACT_STATUS'])

In [ ]:
int_cols = sample_df_poscashb.select_dtypes(int).columns
int_cols

Index(['SK_ID_CURR', 'MONTHS_BALANCE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

In [ ]:
for c in int_cols:
    print(f'{c}: min = {sample_df_poscashb[c].min()}, max = {sample_df_poscashb[c].max()}')

SK_ID_CURR: min = 100002, max = 456255
MONTHS_BALANCE: min = -96, max = -1
NAME_CONTRACT_STATUS: min = 0, max = 5
SK_DPD: min = 0, max = 2914
SK_DPD_DEF: min = 0, max = 344


In [ ]:
sample_df_poscashb['NAME_CONTRACT_STATUS'] = sample_df_poscashb['NAME_CONTRACT_STATUS'].astype(np.int8)

In [ ]:
sample_df_poscashb['MONTHS_BALANCE'] = sample_df_poscashb['MONTHS_BALANCE'].astype(np.int8)

In [ ]:
small_int_cols = ['SK_DPD', 'SK_DPD_DEF']
big_int_cols = ['SK_ID_CURR']

for col in small_int_cols:
    sample_df_poscashb[col] = sample_df_poscashb[col].astype(np.uint16)
    
for col in big_int_cols:
    sample_df_poscashb[col] = sample_df_poscashb[col].astype(np.uint32)

In [ ]:
float_cols = sample_df_poscashb.select_dtypes(float).columns
float_cols

Index(['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE'], dtype='object')

In [ ]:
for c in float_cols:
    print(f'{c}: min = {sample_df_poscashb[c].min()}, max = {sample_df_poscashb[c].max()}')

CNT_INSTALMENT: min = 1.0, max = 72.0
CNT_INSTALMENT_FUTURE: min = 0.0, max = 72.0


In [ ]:
sample_df_poscashb['CNT_INSTALMENT'] = sample_df_poscashb['CNT_INSTALMENT'].astype(np.int8)

In [ ]:
sample_df_poscashb['CNT_INSTALMENT_FUTURE'] = sample_df_poscashb['CNT_INSTALMENT_FUTURE'].astype(np.int8)

In [ ]:
sample_df_poscashb.memory_usage().sum() / 10**6

17.56682

### bureau.csv.zip

In [ ]:
sample_df_bureau.memory_usage() / 10**6

Index                     0.000128
SK_ID_CURR                7.040000
CREDIT_ACTIVE             7.040000
DAYS_CREDIT               7.040000
CREDIT_DAY_OVERDUE        7.040000
DAYS_CREDIT_ENDDATE       7.040000
DAYS_ENDDATE_FACT         7.040000
AMT_CREDIT_MAX_OVERDUE    7.040000
CNT_CREDIT_PROLONG        7.040000
AMT_CREDIT_SUM            7.040000
AMT_CREDIT_SUM_DEBT       7.040000
AMT_CREDIT_SUM_LIMIT      7.040000
AMT_CREDIT_SUM_OVERDUE    7.040000
CREDIT_TYPE               7.040000
DAYS_CREDIT_UPDATE        7.040000
AMT_ANNUITY               7.040000
dtype: float64

In [ ]:
sample_df_bureau.memory_usage().sum() / 10**6

105.600128

In [ ]:
sample_df_bureau.dtypes

SK_ID_CURR                  int64
CREDIT_ACTIVE              object
DAYS_CREDIT                 int64
CREDIT_DAY_OVERDUE          int64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG          int64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE                object
DAYS_CREDIT_UPDATE          int64
AMT_ANNUITY               float64
dtype: object

In [ ]:
sample_df_bureau['CREDIT_ACTIVE'].unique()

array(['Closed', 'Active', 'Sold', 'Bad debt'], dtype=object)

In [ ]:
contract_le = LabelEncoder()
contract_le.fit(sample_df_bureau['CREDIT_ACTIVE'])
sample_df_bureau['CREDIT_ACTIVE'] = contract_le.transform(sample_df_bureau['CREDIT_ACTIVE'])

In [ ]:
#sample_df_bureau['CREDIT_CURRENCY'].unique()

In [ ]:
#contract_le = LabelEncoder()
#contract_le.fit(sample_df_bureau['CREDIT_CURRENCY'])
#sample_df_bureau['CREDIT_CURRENCY'] = contract_le.transform(sample_df_bureau['CREDIT_CURRENCY'])

In [ ]:
contract_le = LabelEncoder()
contract_le.fit(sample_df_bureau['CREDIT_TYPE'])
sample_df_bureau['CREDIT_TYPE'] = contract_le.transform(sample_df_bureau['CREDIT_TYPE'])

In [ ]:
contract_le = LabelEncoder()
contract_le.fit(sample_df_bureau['CREDIT_TYPE'])
sample_df_bureau['CREDIT_TYPE'] = contract_le.transform(sample_df_bureau['CREDIT_TYPE'])

In [ ]:
sample_df_bureau['CREDIT_TYPE'].unique()

array([ 3,  4,  1, 10, 14, 12,  0,  6,  9,  2,  8, 13, 11,  5,  7])

In [ ]:
int_cols = sample_df_bureau.select_dtypes(int).columns
int_cols

Index(['SK_ID_CURR', 'CREDIT_ACTIVE', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'CNT_CREDIT_PROLONG', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE'],
      dtype='object')

In [ ]:
for c in int_cols:
    print(f'{c}: min = {sample_df_bureau[c].min()}, max = {sample_df_bureau[c].max()}')

SK_ID_CURR: min = 100001, max = 456255
CREDIT_ACTIVE: min = 0, max = 3
DAYS_CREDIT: min = -2922, max = 0
CREDIT_DAY_OVERDUE: min = 0, max = 2781
CNT_CREDIT_PROLONG: min = 0, max = 9
CREDIT_TYPE: min = 0, max = 14
DAYS_CREDIT_UPDATE: min = -41946, max = 22


In [ ]:
#sample_df_bureau['CREDIT_CURRENCY'] = sample_df_bureau['CREDIT_CURRENCY'].astype(np.uint8)

In [ ]:
sample_df_bureau['CREDIT_TYPE'] = sample_df_bureau['CREDIT_TYPE'].astype(np.uint8)

In [ ]:
sample_df_bureau['CREDIT_ACTIVE'] = sample_df_bureau['CREDIT_ACTIVE'].astype(np.uint8)

In [ ]:
sample_df_bureau['CNT_CREDIT_PROLONG'] = sample_df_bureau['CNT_CREDIT_PROLONG'].astype(np.uint8)

In [ ]:
small_int_cols = ['DAYS_CREDIT_UPDATE', 'DAYS_CREDIT', 'DAYS_CREDIT_UPDATE']
big_int_cols = ['SK_ID_CURR','CREDIT_DAY_OVERDUE']

for col in small_int_cols:
    sample_df_bureau[col] = sample_df_bureau[col].astype(np.int16)
    
for col in big_int_cols:
    sample_df_bureau[col] = sample_df_bureau[col].astype(np.int32)

In [ ]:
float_cols = sample_df_bureau.select_dtypes(float).columns
float_cols

Index(['DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY'],
      dtype='object')

In [ ]:
for c in float_cols:
    print(f'{c}: min = {sample_df_bureau[c].min()}, max = {sample_df_bureau[c].max()}')

DAYS_CREDIT_ENDDATE: min = -42056.0, max = 31198.0
DAYS_ENDDATE_FACT: min = -42023.0, max = 0.0
AMT_CREDIT_MAX_OVERDUE: min = 0.0, max = 115987185.0
AMT_CREDIT_SUM: min = 0.0, max = 396000000.0
AMT_CREDIT_SUM_DEBT: min = -4705600.32, max = 170100000.0
AMT_CREDIT_SUM_LIMIT: min = -586406.115, max = 4705600.32
AMT_CREDIT_SUM_OVERDUE: min = 0.0, max = 3756681.0
AMT_ANNUITY: min = 0.0, max = 59586682.5


In [ ]:
big_float_cols = ['DAYS_CREDIT_UPDATE', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT','AMT_CREDIT_SUM_LIMIT','AMT_CREDIT_SUM_OVERDUE','AMT_ANNUITY','AMT_CREDIT_MAX_OVERDUE']
small_float_cols = ['DAYS_CREDIT_ENDDATE','DAYS_ENDDATE_FACT']

for col in small_float_cols:
    sample_df_bureau[col] = sample_df_bureau[col].astype(np.int16)
    
for col in big_float_cols:
    sample_df_bureau[col] = sample_df_bureau[col].astype(np.int32)

In [ ]:
sample_df_bureau.memory_usage() / 10**6

Index                     0.000128
SK_ID_CURR                3.520000
CREDIT_ACTIVE             0.880000
DAYS_CREDIT               1.760000
CREDIT_DAY_OVERDUE        3.520000
DAYS_CREDIT_ENDDATE       1.760000
DAYS_ENDDATE_FACT         1.760000
AMT_CREDIT_MAX_OVERDUE    3.520000
CNT_CREDIT_PROLONG        0.880000
AMT_CREDIT_SUM            3.520000
AMT_CREDIT_SUM_DEBT       3.520000
AMT_CREDIT_SUM_LIMIT      3.520000
AMT_CREDIT_SUM_OVERDUE    3.520000
CREDIT_TYPE               0.880000
DAYS_CREDIT_UPDATE        3.520000
AMT_ANNUITY               3.520000
dtype: float64

In [ ]:
sample_df_bureau.memory_usage().sum() / 10**6

39.600128

### bureau_balance.csv.zip

In [ ]:
sample_df_bureaubal.memory_usage().sum() / 10**6

21.120128

In [ ]:
sample_df_bureaubal.dtypes

SK_ID_BUREAU       int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object

In [ ]:
int_cols = sample_df_bureaubal.select_dtypes(int).columns
int_cols

Index(['SK_ID_BUREAU', 'MONTHS_BALANCE'], dtype='object')

In [ ]:
for c in int_cols:
    print(f'{c}: min = {sample_df_bureaubal[c].min()}, max = {sample_df_bureaubal[c].max()}')

SK_ID_BUREAU: min = 5001726, max = 6834663
MONTHS_BALANCE: min = -96, max = 0


In [ ]:
small_float_cols = ['MONTHS_BALANCE']
big_float_cols = ['SK_ID_BUREAU']

for col in small_float_cols:
    sample_df_bureaubal[col] = sample_df_bureaubal[col].astype(np.int8)
    
for col in big_float_cols:
    sample_df_bureaubal[col] = sample_df_bureaubal[col].astype(np.int32)

In [ ]:
sample_df_bureaubal.memory_usage().sum() / 10**6

11.440128

### credit_card_balance.csv.zip

In [ ]:
sample_df_creditcb.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
1,366032,-33,135972.405,135000,0.0,0.0,6750.000,6750.0,6750.0,131812.695,135972.405,135972.405,0.0,0,44.0,Active,0,0
2,257442,-4,181374.750,180000,0.0,0.0,9096.480,9900.0,9900.0,172581.480,180045.855,180045.855,0.0,0,17.0,Active,0,0
3,353507,-4,468008.640,450000,24750.0,24750.0,22285.305,22500.0,22500.0,447956.460,463213.890,463213.890,1.0,1,31.0,Active,0,0
4,322324,-87,72486.135,90000,0.0,0.0,4500.000,7200.0,7200.0,69947.055,72486.135,72486.135,0.0,0,14.0,Active,0,0
5,263926,-23,130329.675,135000,0.0,0.0,6750.000,6750.0,6750.0,125180.595,130329.675,130329.675,0.0,0,51.0,Active,0,0


In [ ]:
sample_df_creditcb.memory_usage().sum() / 10**6

103.8388

In [ ]:
sample_df_creditcb.dtypes

SK_ID_CURR                     int64
MONTHS_BALANCE                 int64
AMT_BALANCE                  float64
AMT_CREDIT_LIMIT_ACTUAL        int64
AMT_DRAWINGS_ATM_CURRENT     float64
AMT_DRAWINGS_CURRENT         float64
AMT_INST_MIN_REGULARITY      float64
AMT_PAYMENT_CURRENT          float64
AMT_PAYMENT_TOTAL_CURRENT    float64
AMT_RECEIVABLE_PRINCIPAL     float64
AMT_RECIVABLE                float64
AMT_TOTAL_RECEIVABLE         float64
CNT_DRAWINGS_ATM_CURRENT     float64
CNT_DRAWINGS_CURRENT           int64
CNT_INSTALMENT_MATURE_CUM    float64
NAME_CONTRACT_STATUS          object
SK_DPD                         int64
SK_DPD_DEF                     int64
dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
contract_le = LabelEncoder()
contract_le.fit(sample_df_poscashb['NAME_CONTRACT_STATUS'])
sample_df_poscashb['NAME_CONTRACT_STATUS'] = contract_le.transform(sample_df_poscashb['NAME_CONTRACT_STATUS'])

In [ ]:
obj_cols = sample_df_creditcb.select_dtypes(object).columns
obj_cols

Index(['NAME_CONTRACT_STATUS'], dtype='object')

In [ ]:
int_cols = sample_df_creditcb.select_dtypes(int).columns
int_cols

Index(['SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL',
       'CNT_DRAWINGS_CURRENT', 'SK_DPD', 'SK_DPD_DEF'],
      dtype='object')

In [ ]:
for c in int_cols:
    print(f'{c}: min = {sample_df_creditcb[c].min()}, max = {sample_df_creditcb[c].max()}')

SK_ID_CURR: min = 100011, max = 456250
MONTHS_BALANCE: min = -96, max = -1
AMT_CREDIT_LIMIT_ACTUAL: min = 0, max = 1350000
CNT_DRAWINGS_CURRENT: min = 0, max = 162
SK_DPD: min = 0, max = 3229
SK_DPD_DEF: min = 0, max = 3229


In [ ]:
#small_int_cols = ['CNT_DRAWINGS_CURRENT','MONTHS_BALANCE']
#medium_int_cols = ['SK_DPD_DEF','SK_DPD_DEF']
#big_int_cols = ['SK_ID_PREV','SK_ID_CURR','AMT_CREDIT_LIMIT_ACTUAL']

#for col in small_int_cols:
 #   sample_df_creditcb[col] = sample_df_creditcb[col].astype(np.int8)

#for col in medium_int_cols:
 #   sample_df_creditcb[col] = sample_df_creditcb[col].astype(np.int16)
    
#for col in big_int_cols:
 #   sample_df_creditcb[col] = sample_df_creditcb[col].astype(np.int32)

In [ ]:
float_cols = sample_df_creditcb.select_dtypes(float).columns
float_cols

Index(['AMT_BALANCE', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT',
       'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT',
       'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL',
       'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM'],
      dtype='object')

In [ ]:
big_float_cols = ['AMT_BALANCE','AMT_DRAWINGS_ATM_CURRENT','AMT_DRAWINGS_CURRENT','AMT_INST_MIN_REGULARITY','AMT_PAYMENT_CURRENT','AMT_PAYMENT_TOTAL_CURRENT','AMT_RECEIVABLE_PRINCIPAL','AMT_RECEIVABLE_PRINCIPAL','AMT_RECIVABLE','AMT_TOTAL_RECEIVABLE','AMT_TOTAL_RECEIVABLE']
small_float_cols = ['CNT_DRAWINGS_ATM_CURRENT','CNT_INSTALMENT_MATURE_CUM']

for col in big_float_cols:
    sample_df_creditcb[col] = sample_df_creditcb[col].astype(np.int32)

for col in small_float_cols:
    sample_df_creditcb[col] = sample_df_creditcb[col].astype(np.int16)

In [ ]:
sample_df_creditcb.memory_usage().sum() / 10**6

71.0476

### previous_application.csv.zip

In [ ]:
sample_df_app.memory_usage().sum() / 10**6

64.0654

In [ ]:
sample_df_app.dtypes

SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_DOWN_PAYMENT               float64
AMT_GOODS_PRICE                float64
WEEKDAY_APPR_PROCESS_START      object
HOUR_APPR_PROCESS_START          int64
FLAG_LAST_APPL_PER_CONTRACT     object
NFLAG_LAST_APPL_IN_DAY           int64
RATE_DOWN_PAYMENT              float64
NAME_CASH_LOAN_PURPOSE          object
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_TYPE_SUITE                 object
NAME_CLIENT_TYPE                object
NAME_GOODS_CATEGORY             object
NAME_PORTFOLIO                  object
NAME_PRODUCT_TYPE               object
CHANNEL_TYPE                    object
SELLERPLACE_AREA                 int64
NAME_SELLER_INDUSTRY            object
CNT_PAYMENT              

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
cat_cols = ['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON','NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE','NAME_SELLER_INDUSTRY','NAME_YIELD_GROUP','PRODUCT_COMBINATION']
label_encoder = {}

# Go through each categorical column
for c in cat_cols:
    # Create a new label encoder for this column
    le = LabelEncoder()
    
    # Fit the label encoder on the column data
    le.fit(sample_df_app[c])
    
    # Transform the column to numeric
    sample_df_app[c] = le.transform(sample_df_app[c])
    
    # Save the label encoder to the label_encoder dictionary, indexed by the column name
    label_encoder[c] = le

In [ ]:
int_cols = sample_df_app.select_dtypes(int).columns
int_cols

Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT',
       'NFLAG_LAST_APPL_IN_DAY', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')

In [ ]:
for c in int_cols:
    print(f'{c}: min = {sample_df_app[c].min()}, max = {sample_df_app[c].max()}')

SK_ID_CURR: min = 100001, max = 456255
NAME_CONTRACT_TYPE: min = 0, max = 2
WEEKDAY_APPR_PROCESS_START: min = 0, max = 6
HOUR_APPR_PROCESS_START: min = 0, max = 23
FLAG_LAST_APPL_PER_CONTRACT: min = 0, max = 0
NFLAG_LAST_APPL_IN_DAY: min = 1, max = 1
NAME_CASH_LOAN_PURPOSE: min = 0, max = 12
NAME_CONTRACT_STATUS: min = 0, max = 0
DAYS_DECISION: min = -2922, max = -2
NAME_PAYMENT_TYPE: min = 0, max = 3
CODE_REJECT_REASON: min = 0, max = 1
NAME_TYPE_SUITE: min = 0, max = 6
NAME_CLIENT_TYPE: min = 0, max = 3
NAME_GOODS_CATEGORY: min = 0, max = 25
NAME_PORTFOLIO: min = 0, max = 2
NAME_PRODUCT_TYPE: min = 0, max = 2
CHANNEL_TYPE: min = 0, max = 5
SELLERPLACE_AREA: min = -1, max = 4000000
NAME_SELLER_INDUSTRY: min = 0, max = 10
NAME_YIELD_GROUP: min = 0, max = 4
PRODUCT_COMBINATION: min = 0, max = 14


In [ ]:
small_int_cols = ['PRODUCT_COMBINATION','NAME_YIELD_GROUP','NAME_YIELD_GROUP','NAME_SELLER_INDUSTRY','CHANNEL_TYPE','NAME_PRODUCT_TYPE','NAME_PRODUCT_TYPE','NAME_PORTFOLIO','NAME_GOODS_CATEGORY','NAME_CLIENT_TYPE','NAME_TYPE_SUITE','CODE_REJECT_REASON','NAME_PAYMENT_TYPE','NAME_CONTRACT_STATUS','NAME_CASH_LOAN_PURPOSE','NFLAG_LAST_APPL_IN_DAY','HOUR_APPR_PROCESS_START','NFLAG_LAST_APPL_IN_DAY','NAME_CONTRACT_TYPE','WEEKDAY_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT']
big_int_cols = ['SK_ID_CURR','SELLERPLACE_AREA','DAYS_DECISION']

for col in small_int_cols:
    sample_df_app[col] = sample_df_app[col].astype(np.int8)
    
for col in big_int_cols:
    sample_df_app[col] = sample_df_app[col].astype(np.uint32)

In [ ]:
sample_df_app.memory_usage().sum() / 10**6

32.49031

# Merging Data Sets

In [ ]:
sample_df_instalpay.shape

(880000, 7)

In [ ]:
sample_df_poscashb.shape

(878341, 7)

In [ ]:
sample_df_bureau.shape

(880000, 15)

In [ ]:
sample_df_bureaubal.shape

(880000, 3)

In [ ]:
sample_df_creditcb.shape

(683150, 18)

In [ ]:
sample_df_app.shape

(228805, 34)

Merging sample_df_app and sample_df_instalpay based on `SK_ID_CURR`

In [ ]:
result = pd.merge(sample_df_app, sample_df_instalpay, how='left', on=['SK_ID_CURR'])

In [ ]:
for c in result.columns:
  num_missing = result[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / result.shape[0]:.2f}%) missing values')

NUM_INSTALMENT_VERSION: 166362 (17.21%) missing values
NUM_INSTALMENT_NUMBER: 166362 (17.21%) missing values
DAYS_INSTALMENT: 166362 (17.21%) missing values
DAYS_ENTRY_PAYMENT: 166362 (17.21%) missing values
AMT_INSTALMENT: 166362 (17.21%) missing values
AMT_PAYMENT: 166362 (17.21%) missing values


Checking for missing values and dropping them.

In [ ]:
result = result.dropna(how='any')

In [ ]:
for c in result.columns:
  num_missing = result[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / result.shape[0]:.2f}%) missing values')

In [ ]:
result.shape

(800299, 40)

In [ ]:
result.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
2,170000,1,5825.205,25192.44,29763.0,1.44,25192.44,6,12,0,1,0.000053,11,0,4294966447,0,0,1,2,20,2,0,4,135,4,6.0,1,7,365243.0,-818.0,-668.0,-668.0,-656.0,0.0,1.0,6.0,-620.0,-611.0,3281.0,3271.0
3,170000,1,5825.205,25192.44,29763.0,1.44,25192.44,6,12,0,1,0.000053,11,0,4294966447,0,0,1,2,20,2,0,4,135,4,6.0,1,7,365243.0,-818.0,-668.0,-668.0,-656.0,0.0,1.0,4.0,-1226.0,-1235.0,3972.0,3972.0
4,170000,1,5825.205,25192.44,29763.0,1.44,25192.44,6,12,0,1,0.000053,11,0,4294966447,0,0,1,2,20,2,0,4,135,4,6.0,1,7,365243.0,-818.0,-668.0,-668.0,-656.0,0.0,1.0,5.0,-650.0,-648.0,3281.0,3275.0
5,170000,1,5825.205,25192.44,29763.0,1.44,25192.44,6,12,0,1,0.000053,11,0,4294966447,0,0,1,2,20,2,0,4,135,4,6.0,1,7,365243.0,-818.0,-668.0,-668.0,-656.0,0.0,1.0,1.0,-1316.0,-1327.0,3972.0,3972.0
6,170000,1,5825.205,25192.44,29763.0,1.44,25192.44,6,12,0,1,0.000053,11,0,4294966447,0,0,1,2,20,2,0,4,135,4,6.0,1,7,365243.0,-818.0,-668.0,-668.0,-656.0,0.0,1.0,1.0,-818.0,-830.0,5825.0,5825.0


In [ ]:
num_past_apps_df = result.groupby('SK_ID_CURR').size().reset_index(name='num_past_apps')

In [ ]:
num_past_apps_df

,SK_ID_CURR,num_past_apps
0,100006,2
1,100007,11
2,100009,16
3,100011,37
4,100012,10
...,...,...
44427,199989,2
44428,199993,7
44429,199997,4
44430,199998,64


In [ ]:
set(num_past_apps_df['SK_ID_CURR']) == set(result['SK_ID_CURR'])

True

Merging result and num past apps df 

In [ ]:
result = pd.merge(result, num_past_apps_df, left_on='SK_ID_CURR', right_on='SK_ID_CURR') 

In [ ]:
for c in result.columns:
  num_missing = result[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / result.shape[0]:.2f}%) missing values')

In [ ]:
result.tail()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,num_past_apps
800294,102212,1,3079.71,29655.0,22846.5,8550.0,29655.0,0,21,0,1,0.296585,11,0,4294965020,0,0,6,0,17,2,0,5,25,2,10.0,1,11,365243.0,-2225.0,-1955.0,-1985.0,-1980.0,0.0,1.0,9.0,-892.0,-900.0,15130.0,15130.0,5
800295,119789,1,9365.04,95220.0,105273.0,0.0,95220.0,6,11,0,1,0.000000,11,0,4294966479,0,0,6,0,6,2,0,2,1600,4,12.0,2,8,365243.0,-786.0,-456.0,-576.0,-568.0,0.0,1.0,4.0,-696.0,-748.0,9365.0,9365.0,4
800296,119789,1,9365.04,95220.0,105273.0,0.0,95220.0,6,11,0,1,0.000000,11,0,4294966479,0,0,6,0,6,2,0,2,1600,4,12.0,2,8,365243.0,-786.0,-456.0,-576.0,-568.0,0.0,1.0,1.0,-786.0,-791.0,9365.0,9365.0,4
800297,119789,1,9365.04,95220.0,105273.0,0.0,95220.0,6,11,0,1,0.000000,11,0,4294966479,0,0,6,0,6,2,0,2,1600,4,12.0,2,8,365243.0,-786.0,-456.0,-576.0,-568.0,0.0,1.0,6.0,-636.0,-690.0,9365.0,9365.0,4
800298,119789,1,9365.04,95220.0,105273.0,0.0,95220.0,6,11,0,1,0.000000,11,0,4294966479,0,0,6,0,6,2,0,2,1600,4,12.0,2,8,365243.0,-786.0,-456.0,-576.0,-568.0,0.0,2.0,4.0,-391.0,-399.0,55215.0,55215.0,4


In [ ]:
result.shape

(800299, 41)

In [ ]:
#result.to_csv(drive_dir + 'result.csv', index = False)

Merging result and sample df poscashb

In [ ]:
result1 = pd.merge(result, sample_df_poscashb, how='left', on=['SK_ID_CURR'])

Checking for missing values and dropping them.

In [ ]:
for c in result1.columns:
  num_missing = result1[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / result1.shape[0]:.2f}%) missing values')

MONTHS_BALANCE: 59107 (1.38%) missing values
CNT_INSTALMENT: 59107 (1.38%) missing values
CNT_INSTALMENT_FUTURE: 59107 (1.38%) missing values
NAME_CONTRACT_STATUS_y: 59107 (1.38%) missing values
SK_DPD: 59107 (1.38%) missing values
SK_DPD_DEF: 59107 (1.38%) missing values


In [ ]:
result1 = result1.dropna(how='any')

In [ ]:
for c in result1.columns:
  num_missing = result1[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / result1.shape[0]:.2f}%) missing values')

In [ ]:
result1.shape

(4231647, 47)

Merging result1 and sample df bureau

In [ ]:
result2 = pd.merge(result1, sample_df_bureau, how='left', on=['SK_ID_CURR'])

In [ ]:
result2.to_csv(drive_dir + 'result2.csv', index = False )